In [55]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `C:\Users\Ferdi`


In [56]:
using HiGHS
using JuMP
#using Plots
#using XLSX
#using DataFrames
#using Ipopt
#using Complementarity

In [57]:
T=3 #Number of time periods
W=3 #Number of scenarios
probW=[0.2,0.3,0.5] #Probability of scenarios

EGmax=5 #max energy production Mw at VPP
EGmin=1 #min ""

ELmax=2   #max energy consumption
ELmin=0.5 #min ""
EL_floor= 2 # energy must be consumend in T

PdSmax=0.5 #max discharging power
PcSmax=0.3 #max charging power
ndS=0.8 #loss by discharging
ncS=0.8 #loss by charging 
ESmax=1   #max energy storage
ESmin=0.2 #min "" 
E_S0=0.6 #energy storage t=0

PWexcept=[3.3,3.0,3.00] #forcast power production for periods
PW=[2.5 6.0 2.0; #power production at t in w
    4.0 4.0 1.1;
    6.0 3.5 1.5]

yD=[20,80,45] #market price

Utility=[10 20 10; #utility at t in w
    50 20 20;
    10 30 10]
Cost = [5 10 10; #cost of production at t in w
    15 20 15;
    30 15 10]

3×3 Matrix{Int64}:
  5  10  10
 15  20  15
 30  15  10

In [58]:
m3 = Model(HiGHS.Optimizer)


@variable(m3, v[t in 1:T, w in 1:W], Bin)
@variable(m3, E_G[t in 1:T, w in 1:W])
@variable(m3, E_L[t in 1:T, w in 1:W])
@variable(m3, E_S[t in 1:T, w in 1:W])
@variable(m3, Pd_S[t in 1:T, w in 1:W] >= 0)
@variable(m3, Pc_S[t in 1:T, w in 1:W] >= 0)
@variable(m3, P_D[t in 1:T])
@variable(m3, PWcurt[t in 1:T, w in 1:W] >= 0)


3×3 Matrix{VariableRef}:
 PWcurt[1,1]  PWcurt[1,2]  PWcurt[1,3]
 PWcurt[2,1]  PWcurt[2,2]  PWcurt[2,3]
 PWcurt[3,1]  PWcurt[3,2]  PWcurt[3,3]

In [59]:
@constraints(m3, begin
        VPPenergyproductionlow[t in 1:T, w in 1:W], v[t,w]*EGmin <= E_G[t,w]
        VPPenergyproductionup[t in 1:T, w in 1:W], E_G[t,w] <= v[t,w]*EGmax
        loadenergyconsumption[t in 1:T, w in 1:W], ELmin <= E_L[t,w] <= ELmax
        floorenergyconsumption[t in 1:T, w in 1:W], EL_floor <= sum(E_L[t,w] for t in 1:T)
        storagedischarging[t in 1:T, w in 1:W], Pd_S[t,w] <= PdSmax
        storagecharging[t in 1:T, w in 1:W], Pc_S[t,w] <= PcSmax
        storagelevel1[t in 1:1, w in 1:W], E_S[t,w] == E_S0 + ncS*Pc_S[t,w] - (1/ndS)*Pd_S[t,w]
        storagelevel[t in 2:T, w in 1:W], E_S[t,w] == E_S[t-1,w] + ncS*Pc_S[t,w] - (1/ndS)*Pd_S[t,w]
        storagelevelcapacity[t in 1:T, w in 1:W], ESmin <= E_S[t,w] <=ESmax
        PVproductioncurtailment[t in 1:T, w in 1:W], PWcurt[t,w] <= PW[t,w]
        energybalance[t in 1:T, w in 1:W], E_G[t,w] + PW[t,w] - PWcurt[t,w] + Pd_S[t,w] == E_L[t,w] + Pc_S[T,w] + P_D[t]
    end)

(ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}[VPPenergyproductionlow[1,1] : v[1,1] - E_G[1,1] <= 0 VPPenergyproductionlow[1,2] : v[1,2] - E_G[1,2] <= 0 VPPenergyproductionlow[1,3] : v[1,3] - E_G[1,3] <= 0; VPPenergyproductionlow[2,1] : v[2,1] - E_G[2,1] <= 0 VPPenergyproductionlow[2,2] : v[2,2] - E_G[2,2] <= 0 VPPenergyproductionlow[2,3] : v[2,3] - E_G[2,3] <= 0; VPPenergyproductionlow[3,1] : v[3,1] - E_G[3,1] <= 0 VPPenergyproductionlow[3,2] : v[3,2] - E_G[3,2] <= 0 VPPenergyproductionlow[3,3] : v[3,3] - E_G[3,3] <= 0], ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}[VPPenergyproductionup[1,1] : -5 v[1,1] + E_G[1,1] <= 0 VPPenergyproductionup[1,2] : -5 v[1,2] + E_G[1,2] <= 0 VPPenergyproductionup[1,3] : -5 v[1,3] + E_G[1,3] <= 0; VPPenergyproductionup[2,1] : -5 v[2,1] + E_G[2,1

In [60]:
@objective(m3, Max,(sum(yD[t]*P_D[t] + sum(probW[w] * (Utility[t,w]*E_L[t,w] - Cost[t,w]*E_G[t,w]) for w in 1:W) for t in 1:T)))
optimize!(m3)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
45 rows, 66 cols, 150 nonzeros
39 rows, 66 cols, 132 nonzeros

Solving MIP model with:
   39 rows
   66 cols (9 binary, 0 integer, 0 implied int., 57 continuous)
   132 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   1030.5          -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   812.418         650.986           24.80%        0      0      0        21     0.0s
 C       0       0         0   0.00%   809.53          669.034           21.00%       39      6      0        34     0.0s
 T       0       0         0   0.00%   809.53          809.53             0.00%       39      6      0        34     0.0s



In [61]:
objective_value(m3)

809.53